In [1]:
#basic imports
import numpy as np
import pandas as pd

#visualization libraries
import cufflinks as cf
cf.go_offline()

import plotly.graph_objs as go
import plotly.tools as tls
import plotly.io as pio
from plotly.offline import iplot

import plotly.io as pio

In [2]:
exercises = pd.read_csv('../fitnotes/exercises.csv')

In [3]:
volume = exercises[(exercises.category == 'Chest') & (exercises.routine == 'Princess Diana Gains Guide')].groupby('date')[['volume']].sum()

In [4]:
volume.head()

,volume
date,
2018-05-18,6011.0
2018-05-23,6779.0
2018-05-31,4590.0
2018-06-08,4747.0
2018-06-14,8079.0


In [5]:
sleep = pd.read_csv('../fitbit/sleep.csv')

In [6]:
sleep.duration = sleep.duration.div(1000).div(3600)

In [7]:
sleep = sleep.groupby('dateOfSleep', as_index = False).duration.sum()

In [8]:
sleep.dateOfSleep = pd.to_datetime(sleep.dateOfSleep) + pd.Timedelta('1 day')

In [9]:
sleep.set_index('dateOfSleep', inplace = True)

In [10]:
test = volume.join(sleep).query('duration > 0')

In [11]:
test.index = pd.to_datetime(test.index)

In [12]:
test

,volume,duration
date,,
2018-05-23,6779.0,8.750000
2018-06-08,4747.0,10.933333
2018-06-14,8079.0,6.150000
2018-06-20,1060.0,10.466667
2018-06-27,5652.0,7.216667
2018-07-04,9372.0,4.100000
2018-07-12,4721.0,7.050000
2018-07-19,9224.0,5.716667
2018-07-25,9712.0,5.883333


In [13]:
subplots = tls.make_subplots(rows=2, cols=2, 
                              specs=[[{}, {}], 
                                     [{}, {}]
                                    ],
                                 subplot_titles = ['Daily', 'Weekly', 'Monthly', 'Yearly'],
                              print_grid=True)

f = go.FigureWidget(subplots)

samples = ['D', 'W', 'M', 'A']

rows = [1, 1, 2, 2]
cols = [1, 2, 1, 2]

for sample, row, col in zip(samples, rows, cols):
    sampled = test.resample(sample).sum().query('volume > 0')
    f.add_scatter(x = sampled.duration, 
                  y = sampled.volume, 
                  mode = 'markers', 
                  marker = {'opacity': .5, 
                            'color': 'black',
                            'size': 5,
                            'line': {'color': 'black', 'width': .5}
                           },
                  showlegend = False, 
                  row = row, 
                  col = col)
    
    f.add_histogram2dcontour(x = sampled.duration, 
                  y = sampled.volume, 
                  showlegend = False, 
                      showscale = False,
                  row = row, 
                  col = col)
    
        


f.layout.title = '<b>Less Sleep Means Less Weight Lifted on a Weekly and Monthly Basis</b><br><i>Total Sleep vs. Total Volume Lifted'

f.layout.yaxis.title = 'Volume Lifted (lbs)'

f.layout.xaxis3.title = 'Hours of Sleep the Previous Night'

# set colors
#f.layout.plot_bgcolor = '#E5E5E5'
#f.layout.paper_bgcolor = '#E5E5E5'

f.layout.images = [dict(
        source="https://raw.githubusercontent.com/WilliamGreenlaw/Logos/master/signature_min.png",
        xref="paper", yref="paper",
        x=1.05, y=1.10,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
      )]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



In [14]:
test.head()

,volume,duration
date,,
2018-05-23,6779.0,8.750000
2018-06-08,4747.0,10.933333
2018-06-14,8079.0,6.150000
2018-06-20,1060.0,10.466667
2018-06-27,5652.0,7.216667


In [15]:
f.layout.xaxis.showgrid = True
f.layout.yaxis.showgrid = True

In [16]:
f

FigureWidget({
    'data': [{'marker': {'color': 'black', 'line': {'color': 'black', 'width': 0.5}, 'opacity':…

In [17]:
pio.write_image(f, 'sleep.svg', width = 1500, height = 500)
pio.write_image(f, 'sleep.png', width = 1500, height = 500)